# Use AutoAI to predict credit risk with `ibm-watson-machine-learning`

This notebook demonstrates how to deploy in Watson Machine Learning service an AutoAI model created in `Generated Scikit-learn Notebook`  
which is composed
during autoai experiments (in order to learn more about AutoAI experiments go to [experiments/autoai](https://github.com/IBM/watson-machine-learning-samples/tree/master/cloud/notebooks/python_sdk/experiments/autoai)).

Some familiarity with bash is helpful. This notebook uses Python.


## Learning goals

The learning goals of this notebook are:

-  Working with the Watson Machine Learning instance
-  Online deployment of AutoAI model
-  Scoring data using deployed model


## Contents

This notebook contains the following parts:

1. [Setup](#setup)
2. [Model upload](#upload) 
3. [Web service creation](#deploy)
4. [Scoring](#score)
5. [Clean up](#cleanup)
6. [Summary and next steps](#summary)

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href=" https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-service-instance.html" target="_blank" rel="noopener no referrer">here</a>).

### Connection to WML

Authenticate the Watson Machine Learning service on IBM Cloud. You need to provide platform `api_key` and instance `location`.

You can use [IBM Cloud CLI](https://cloud.ibm.com/docs/cli/index.html) to retrieve platform API Key and instance location.

API Key can be generated in the following way:
```
ibmcloud login
ibmcloud iam api-key-create API_KEY_NAME
```

In result, get the value of `api_key` from the output.


Location of your WML instance can be retrieved in the following way:
```
ibmcloud login --apikey API_KEY -a https://cloud.ibm.com
ibmcloud resource service-instance WML_INSTANCE_NAME
```

In result, get the value of `location` from the output.

**Tip**: Your `Cloud API key` can be generated by going to the [**Users** section of the Cloud console](https://cloud.ibm.com/iam#/users). From that page, click your name, scroll down to the **API Keys** section, and click **Create an IBM Cloud API key**. Give your key a name and click **Create**, then copy the created key and paste it below. You can also get a service specific url by going to the [**Endpoint URLs** section of the Watson Machine Learning docs](https://cloud.ibm.com/apidocs/machine-learning).  You can check your instance location in your  <a href="https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance details.

You can also get service specific apikey by going to the [**Service IDs** section of the Cloud Console](https://cloud.ibm.com/iam/serviceids).  From that page, click **Create**, then copy the created key and paste it below.

**Action**: Enter your `api_key` and `location` in the following cell.

In [ ]:
api_key = 'PASTE YOUR PLATFORM API KEY HERE'
location = 'PASTE YOUR INSTANCE LOCATION HERE'

In [ ]:
wml_credentials = {
    "apikey": api_key,
    "url": 'https://' + location + '.ml.cloud.ibm.com'
}

### Install and import the `ibm-watson-machine-learning` package
**Note:** `ibm-watson-machine-learning` documentation can be found <a href="http://ibm-wml-api-pyclient.mybluemix.net/" target="_blank" rel="noopener no referrer">here</a>.

In [ ]:
!pip install -U ibm-watson-machine-learning

In [ ]:
from ibm_watson_machine_learning import APIClient

client = APIClient(wml_credentials)

### Working with spaces

First, create a space that will be used for your work. If you do not have space already created, you can use [Deployment Spaces Dashboard](https://dataplatform.cloud.ibm.com/ml-runtime/spaces) to create one.

- Click New Deployment Space
- Create an empty space
- Select Cloud Object Storage
- Select Watson Machine Learning instance and press Create
- Copy `space_id` and paste it below

**Tip**: You can also use SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: Assign space ID below

In [ ]:
space_id = 'PASTE YOUR SPACE ID HERE'

You can use `list` method to print all existing spaces.

In [ ]:
client.spaces.list(limit=10)

To be able to interact with all resources available in Watson Machine Learning, you need to set **space** which you will be using.

In [ ]:
client.set.default_space(space_id)

<a id="upload"></a>
## 2. Upload model

In this section you will learn how to upload the model to the Cloud.


#### Download the data as an pandas DataFrame and AutoAI saved as scikit pipeline model using `wget`.
**Hint**: To install required packages, execute command `!pip install pandas wget numpy`.

We can extract model from the executed AutoAI experiment using `ibm-watson-machine-learning` with following command: `experiment.optimizer(...).get_pipeline(astype='sklearn')`.

In [ ]:
!pip install pandas wget numpy

In [ ]:
import os, wget
import pandas as pd
import numpy as np

filename = 'german_credit_data_biased_training.csv'
url = 'https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/data/credit_risk/german_credit_data_biased_training.csv'
if not os.path.isfile(filename):
    wget.download(url)

model_name = "model.pickle"
url = 'https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/models/autoai/credit-risk/model.pickle'
if not os.path.isfile(model_name):
    wget.download(url)

credit_risk_df = pd.read_csv(filename)
X = credit_risk_df.drop(['Risk'], axis=1)
y = credit_risk_df['Risk']

credit_risk_df.head()

#### Custom software_specification
Create new software specification based on default Python environment extended by autoai-libs package.

In [ ]:
base_sw_spec_uid = client.software_specifications.get_uid_by_name("runtime-22.2-py3.10")

In [ ]:
url = 'https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/configs/config.yaml'
if not os.path.isfile('config.yaml'):
    wget.download(url)

In [ ]:
!cat config.yaml

`config.yaml` file describes details of package extention. Now you need to store new package extention with APIClient.

In [ ]:
meta_prop_pkg_extn = {
    client.package_extensions.ConfigurationMetaNames.NAME: "scikt with autoai-libs",
    client.package_extensions.ConfigurationMetaNames.DESCRIPTION: "Extension for autoai-libs",
    client.package_extensions.ConfigurationMetaNames.TYPE: "conda_yml"
}

pkg_extn_details = client.package_extensions.store(meta_props=meta_prop_pkg_extn, file_path="config.yaml")
pkg_extn_uid = client.package_extensions.get_uid(pkg_extn_details)
pkg_extn_url = client.package_extensions.get_href(pkg_extn_details)

#### Create new software specification and add created package extention to it.

In [ ]:
meta_prop_sw_spec = {
    client.software_specifications.ConfigurationMetaNames.NAME: "Mitigated AutoAI bases on scikit spec",
    client.software_specifications.ConfigurationMetaNames.DESCRIPTION: "Software specification for scikt with autoai-libs",
    client.software_specifications.ConfigurationMetaNames.BASE_SOFTWARE_SPECIFICATION: {"guid": base_sw_spec_uid}
}

sw_spec_details = client.software_specifications.store(meta_props=meta_prop_sw_spec)
sw_spec_uid = client.software_specifications.get_uid(sw_spec_details)

client.software_specifications.add_package_extension(sw_spec_uid, pkg_extn_uid)

#### Get the details of created software specification

In [ ]:
client.software_specifications.get_details(sw_spec_uid)

#### Load the AutoAI model saved as `scikit-learn` pipeline.

Depending on estimator type in autoai model pipeline may consist models from following frameworks:
 - `xgboost`
 - `lightgbm`
 - `scikit-learn`

In [ ]:
from joblib import load

pipeline = load(model_name)

#### Store the model

In [ ]:
model_props = {
    client.repository.ModelMetaNames.NAME: "AutoAI model",
    client.repository.ModelMetaNames.TYPE: 'scikit-learn_1.0',
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sw_spec_uid
    
}
feature_vector = X.columns

In [ ]:
published_model = client.repository.store_model(
    model=pipeline, 
    meta_props=model_props,
    training_data=X.values,
    training_target=y.values,
    feature_names=feature_vector,
    label_column_names=['Risk']
)

In [ ]:
published_model_uid = client.repository.get_model_id(published_model)

#### Get model details

In [ ]:
client.repository.get_details(published_model_uid)

**Note:** You can see that model is successfully stored in Watson Machine Learning Service.

In [ ]:
client.repository.list_models()

<a id="deploy"></a>
## 3. Create online deployment
You can use commands bellow to create online deployment for stored model (web service).

In [ ]:
metadata = {
    client.deployments.ConfigurationMetaNames.NAME: "Deployment of AutoAI model.",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

created_deployment = client.deployments.create(published_model_uid, meta_props=metadata)

Get deployment id.

In [ ]:
deployment_id = client.deployments.get_uid(created_deployment)
print(deployment_id)

<a id="score"></a>
## 4. Scoring

You can send new scoring records to web-service deployment using `score` method.

In [ ]:
values = X.values

scoring_payload = {
    "input_data": [{
        'values': values[:5]
    }]
}

In [ ]:
predictions = client.deployments.score(deployment_id, scoring_payload)
predictions

<a id="cleanup"></a>
## 5. Clean up   

If you want to clean up all created assets:
- experiments
- trainings
- pipelines
- model definitions
- models
- functions
- deployments

see the steps in this sample [notebook](https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/instance-management/Machine%20Learning%20artifacts%20management.ipynb).

<a id="summary"></a>
## 6. Summary and next steps     

 You successfully completed this notebook! You learned how to use Watson Machine Learning for AutoA model deployment and scoring. Check out our [Online Documentation](https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/welcome-main.html) for more samples, tutorials, documentation, how-tos, and blog posts. 

### Author

**Jan Sołtysik** Intern in Watson Machine Learning.

Copyright © 2020, 2021 IBM. This notebook and its source code are released under the terms of the MIT License.